In [1]:
#pip install riotwatcher

In [2]:
import requests
import json
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
import time
import datetime

In [3]:
api_key = 'RGAPI-fc004fa6-2cbe-4d70-9b27-4a263be5032b'
watcher = LolWatcher(api_key)
my_region = 'na1'

me = watcher.summoner.by_name(my_region, 'Yozu')
print(me)

{'id': 'X9Fxahe-EAZ-mkCBIeInS6YB8B3G9XyClC7WfNCFZYsKHNI', 'accountId': 'rbdQhRidH5RU6eopOwpIH1EF7K4Fmda_lIM_-G75MSZcaw', 'puuid': '0jAFLz_ywYJrVhlfraL6BYzaErBnnheZ-cy_D26CZXI_fQmuddZ-QDZvlkDLDLtX-s5_CHp7F4_JoA', 'name': 'Yozu', 'profileIconId': 4100, 'revisionDate': 1650082174723, 'summonerLevel': 414}


In [4]:
my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])
pprint.pprint(my_ranked_stats)

[{'freshBlood': True,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '4d82a824-4849-4c68-8589-a9d6b5754e91',
  'leaguePoints': 85,
  'losses': 11,
  'queueType': 'RANKED_FLEX_SR',
  'rank': 'IV',
  'summonerId': 'X9Fxahe-EAZ-mkCBIeInS6YB8B3G9XyClC7WfNCFZYsKHNI',
  'summonerName': 'Yozu',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 25},
 {'freshBlood': True,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '697ed7b8-6d1d-3295-a91a-ad04b9a7d1eb',
  'leaguePoints': 614,
  'losses': 187,
  'queueType': 'RANKED_SOLO_5x5',
  'rank': 'I',
  'summonerId': 'X9Fxahe-EAZ-mkCBIeInS6YB8B3G9XyClC7WfNCFZYsKHNI',
  'summonerName': 'Yozu',
  'tier': 'GRANDMASTER',
  'veteran': False,
  'wins': 195}]


In [5]:
total_games_plays = my_ranked_stats[0]['wins'] + my_ranked_stats[0]['losses']

my_matches = []

my_matches.extend(watcher.match.matchlist_by_puuid('americas', me['puuid'], 0, 100))
pprint.pprint(my_matches)

['NA1_4279193474',
 'NA1_4278903848',
 'NA1_4278931082',
 'NA1_4278230862',
 'NA1_4278280338',
 'NA1_4277524345',
 'NA1_4277520341',
 'NA1_4277452351',
 'NA1_4277376164',
 'NA1_4277349568',
 'NA1_4276780151',
 'NA1_4276735269',
 'NA1_4276122625',
 'NA1_4276122605',
 'NA1_4276122597',
 'NA1_4276658042',
 'NA1_4276362177',
 'NA1_4276306861',
 'NA1_4276323241',
 'NA1_4276280343',
 'NA1_4275761079',
 'NA1_4275739575',
 'NA1_4275737808',
 'NA1_4275705156',
 'NA1_4275713023',
 'NA1_4275696390',
 'NA1_4275672528',
 'NA1_4275206394',
 'NA1_4275193714',
 'NA1_4275159965',
 'NA1_4275105797',
 'NA1_4275102799',
 'NA1_4275109967',
 'NA1_4275057161',
 'NA1_4274371594',
 'NA1_4273694999',
 'NA1_4273699919',
 'NA1_4272790030',
 'NA1_4272755050',
 'NA1_4272709111',
 'NA1_4272672703',
 'NA1_4272555218',
 'NA1_4272125028',
 'NA1_4272160861',
 'NA1_4272076208',
 'NA1_4272062407',
 'NA1_4272047834',
 'NA1_4270984601',
 'NA1_4270970161',
 'NA1_4270907448',
 'NA1_4270933112',
 'NA1_4270909860',
 'NA1_427083

In [8]:
#my_matches = watcher.match.matchlist_by_puuid('americas', me['puuid'])
table = []
for my_match in my_matches:
    match_detail = watcher.match.by_id('americas', my_match)

    info = {}
    team_total_damages_to_champs = 0
    
    try:
    
        if match_detail["info"]['gameMode'] == 'CLASSIC':
            for participant in match_detail["info"]["participants"]:
                if participant["summonerName"] == me['name']:
                    info["ChampionName"] = participant['championName']
                    info["ChampionLevel"] = participant['champLevel']
                    info["Lane"] = participant['lane']
                    info["Kills"] = participant['kills']
                    info["Deaths"] = participant['deaths']
                    info["Assists"] = participant['assists']
                    info["Gold_Earned"] = participant['goldEarned']
                    info["Total_Minions_Killed"] = participant['totalMinionsKilled']
                    info["Total_damages_to_champs"] = participant['totalDamageDealtToChampions']
                    info["Win/Lose"] = participant['win']
                    info["TeamBaronKills"] = participant['challenges']['teamBaronKills']
                    info["FirstBloodAssist"] = participant['firstBloodAssist']
                    info["FirstBloodKill"] = participant['firstBloodKill']
                    info["FirstTowerAssist"] = participant['firstTowerAssist']
                    info["FirstTowerKill"] = participant['firstTowerKill']
                    info["InhibitorKills"] = participant['inhibitorKills']
                    info["InhibitorTakedowns"] = participant['inhibitorTakedowns']
                    info["InhibitorsLost"] = participant['inhibitorsLost']
                    info["TurretPlatesTaken"] = participant['challenges']['turretPlatesTaken']



            for participant in match_detail["info"]["participants"]:
                if participant['win'] == info["Win/Lose"]:
                    team_total_damages_to_champs += participant['totalDamageDealtToChampions']

            info["TotalDamageDealtToChampions_Percentage"] = round(info["Total_damages_to_champs"] / team_total_damages_to_champs * 100, 2)
            
            date = match_detail['info']['gameCreation']
            dt = datetime.datetime.fromtimestamp(date/1000)
            info["Date"] = f'{dt.month}/{dt.day}/{dt.year}'
            
            table.append(info)
            time.sleep(0.1)

    except:
        print('Error')

pprint.pprint(table)


[{'Assists': 8,
  'ChampionLevel': 9,
  'ChampionName': 'Renata',
  'Date': '4/16/2022',
  'Deaths': 5,
  'FirstBloodAssist': False,
  'FirstBloodKill': True,
  'FirstTowerAssist': False,
  'FirstTowerKill': False,
  'Gold_Earned': 5949,
  'InhibitorKills': 0,
  'InhibitorTakedowns': 0,
  'InhibitorsLost': 1,
  'Kills': 1,
  'Lane': 'BOTTOM',
  'TeamBaronKills': 0,
  'TotalDamageDealtToChampions_Percentage': 7.71,
  'Total_Minions_Killed': 4,
  'Total_damages_to_champs': 4386,
  'TurretPlatesTaken': 1,
  'Win/Lose': False},
 {'Assists': 14,
  'ChampionLevel': 13,
  'ChampionName': 'Janna',
  'Date': '4/15/2022',
  'Deaths': 1,
  'FirstBloodAssist': False,
  'FirstBloodKill': False,
  'FirstTowerAssist': False,
  'FirstTowerKill': False,
  'Gold_Earned': 8010,
  'InhibitorKills': 0,
  'InhibitorTakedowns': 0,
  'InhibitorsLost': 5,
  'Kills': 2,
  'Lane': 'BOTTOM',
  'TeamBaronKills': 0,
  'TotalDamageDealtToChampions_Percentage': 8.34,
  'Total_Minions_Killed': 13,
  'Total_damages_to_

In [9]:
df = pd.DataFrame(table)
df.head()

,ChampionName,ChampionLevel,Lane,Kills,Deaths,Assists,Gold_Earned,Total_Minions_Killed,Total_damages_to_champs,Win/Lose,...,FirstBloodAssist,FirstBloodKill,FirstTowerAssist,FirstTowerKill,InhibitorKills,InhibitorTakedowns,InhibitorsLost,TurretPlatesTaken,TotalDamageDealtToChampions_Percentage,Date
0,Renata,9,BOTTOM,1,5,8,5949,4,4386,False,...,False,True,False,False,0,0,1,1,7.71,4/16/2022
1,Janna,13,BOTTOM,2,1,14,8010,13,5155,False,...,False,False,False,False,0,0,5,1,8.34,4/15/2022
2,Renata,10,BOTTOM,2,2,6,5374,13,4555,False,...,False,False,False,False,0,0,1,1,11.48,4/15/2022
3,FiddleSticks,18,TOP,7,6,17,13581,187,32915,True,...,False,False,False,False,0,0,3,2,15.61,4/14/2022
4,Ziggs,11,NONE,5,5,3,6919,104,10998,False,...,False,False,False,True,0,0,0,5,37.68,4/14/2022


In [10]:
df.to_csv('Resources/Yozu_Visuals.csv')

In [12]:
CookiesandCream_df = pd.read_csv("Resources/CookiesandCream_Visuals.csv")
CookiesandCream_df = CookiesandCream_df.drop(['Unnamed: 0'], axis = 1)
CookiesandCream_df["rank"] = "Gold"

trade_df = pd.read_csv("Resources/I_will_trade_Visuals.csv")
trade_df  = trade_df .drop(['Unnamed: 0'], axis = 1)
trade_df["rank"] = "Challenger"

iron_4 = pd.read_csv("Resources/iron_4_Visuals.csv")
iron_4  = iron_4.drop(['Unnamed: 0'], axis = 1)
iron_4["rank"] = "Iron"

MeleezaNoober_df = pd.read_csv("Resources/MeleezaNoober_Visuals.csv")
MeleezaNoober_df = MeleezaNoober_df.drop(['Unnamed: 0'], axis = 1)
MeleezaNoober_df["rank"] = "Silver"

Razei_df = pd.read_csv("Resources/Razei_Visuals.csv")
Razei_df = Razei_df.drop(['Unnamed: 0'], axis = 1)
Razei_df["rank"] = "Platinum"

soldr_df = pd.read_csv("Resources/Soldr_Visuals.csv")
soldr_df  = soldr_df.drop(['Unnamed: 0'], axis = 1)
soldr_df["rank"] = "Diamond"

syptime_df = pd.read_csv("Resources/syptime_Visuals.csv")
syptime_df  = syptime_df.drop(['Unnamed: 0'], axis = 1)
syptime_df["rank"] = "Bronze"

xRoyal_Master_df = pd.read_csv("Resources/xRoyal_Visuals.csv")
xRoyal_Master_df  = xRoyal_Master_df .drop(['Unnamed: 0'], axis = 1)
xRoyal_Master_df["rank"] = "Master"

Yozu_Grandmaster_df = pd.read_csv("Resources/Yozu_Visuals.csv")
Yozu_Grandmaster_df = Yozu_Grandmaster_df.drop(['Unnamed: 0'], axis = 1)
Yozu_Grandmaster_df["rank"] = "Grandmaster"

In [15]:
frames = [Razei_df, CookiesandCream_df, trade_df, iron_4, 
          MeleezaNoober_df, soldr_df, syptime_df, xRoyal_Master_df, Yozu_Grandmaster_df]
df_final = pd.concat(frames)
df_final.to_csv("Resources/LOL_data_date.csv")